In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
import re

In [0]:
# Definindo o catálogo e schemas
CATALOGO = "catalogo_energia"
SCHEMA_BRONZE = "bronze"
SCHEMA_SILVER = "silver"

In [0]:
# Lendo as tabelas Delta da camada Bronze
df_geral_bronze = spark.read.table(f"{CATALOGO}.{SCHEMA_BRONZE}.consumo_geral")
df_limpa_bronze = spark.read.table(f"{CATALOGO}.{SCHEMA_BRONZE}.energia_limpa")

print(f"Total de linhas (Geral): {df_geral_bronze.count()}")
print(f"Total de linhas (Limpa): {df_limpa_bronze.count()}")

In [0]:
def limpar_e_padronizar(df):
    
    # 1. Tratamento de Nulos em Colunas Numéricas
    # Substituir nulos por 0 nas colunas que serão usadas em cálculos.
    colunas_numericas = ["consumo_em_kw", "valor_da_conta", "valor_de_imposto"]
    df = df.fillna(0, subset=colunas_numericas)
    
    # 2. Padronização de Colunas de Texto
    # Garante consistência de agrupamento para a análise.
    colunas_texto_chave = ["regiao", "cidade", "bairro", "tipo_de_cliente"]
    
    for col in colunas_texto_chave:
        # F.trim: remove espaços antes/depois; F.initcap: 'sao paulo' -> 'Sao Paulo'.
        df = df.withColumn(col, F.initcap(F.trim(F.col(col))))

    # 3. Conversão de Tipos (Cast) - Garantindo DoubleType para precisão
    for col in colunas_numericas:
        df = df.withColumn(col, F.col(col).cast(DoubleType()))
        
    return df

# Aplicando a função em ambos os DataFrames
df_geral_limpo = limpar_e_padronizar(df_geral_bronze)
df_limpa_limpo = limpar_e_padronizar(df_limpa_bronze)